In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from config import *
from load_data import fetch_data, fetch_trajectory
from match_rounds import find_match_round_dtw, find_match_round_dtw_kmp
from utils import compute_stat, interplate_and_align

# DataLoader

In [2]:
import os

file_list = os.listdir(DATA_DIR)
file_list = list(filter(lambda x: "AD" in x, file_list))
file_list

['24071512_AD.csv',
 '24070902_AD.csv',
 '24071009_AD.csv',
 '24072825_AD.csv',
 '24070904_AD.csv',
 '24071617_AD.csv',
 '24071721_AD.csv',
 '24071619_AD.csv',
 '24071615_AD.csv',
 '24071011_AD.csv',
 '24070906_AD.csv',
 'Test_AD.csv',
 '24071513_AD.csv',
 '24072926_AD.csv',
 '24071008_AD.csv',
 '24070901_AD.csv',
 '24072023_AD.csv',
 '24071616_AD.csv',
 '24070905_AD.csv',
 '24070907_AD.csv',
 '24071010_AD.csv',
 '24071618_AD.csv']

In [3]:
drop_list = ['pingpang.csv', 'tennis.csv', '.DS_Store', 'ControlGroupInfo.xlsx',]

In [4]:
all_data = fetch_data(DATA_DIR, file_list, drop_list)
ball_data = fetch_trajectory(DATA_DIR)

# Feature Extraction

## Saccade Feature

In [5]:
def compute_saccade_path_lite(df:pd.DataFrame):
    _speeds = [0]
    _angles = [0]
    indices = list(df.index)
    _dura = (1 * EYE_SAMPLE_TIME ) / 1000.0
    df["Screen.x"] = df["Screen.x"] * VR_SCALE
    df["Screen.y"] = df["Screen.y"] * VR_SCALE

    for _i in range(1, len(indices)):
        row_1 = df.loc[indices[_i - 1], :]
        row_2 = df.loc[indices[_i], :]

        _dist = np.sqrt((row_1["Screen.x"] - row_2["Screen.x"])**2 + (row_1["Screen.y"] - row_2["Screen.y"])**2)
        _angle = np.arctan(_dist / VR_ZDIST) / np.pi * 180

        _angles.append(_angle)
        _speeds.append(np.divide(_angle, _dura))

    return _speeds, _angles

## Trajectory Features

In [6]:

def extract_saccade_features_lite(round_index, data:pd.DataFrame, ball_data_df:pd.DataFrame):
    _fea = {}

    _speeds, _angles = compute_saccade_path_lite(data)
    _fea = {**_fea, **compute_stat("SaccadeSpeed", _speeds)}
    _fea = {**_fea, **compute_stat("SaccadeAngel", _angles)}

    round_start_index = ball_data_df[ball_data_df["round"]==round_index].index[0]
    round_dura = ball_data_df[ball_data_df["round"]==round_index].shape[0]
    eye_start_index = data.index[0]
    _fea["SaccadeDelay"] = (eye_start_index-round_start_index) * EYE_SAMPLE_TIME
    _fea["SaccadeDelayPercent"] = ((eye_start_index-round_start_index) / round_dura) * EYE_SAMPLE_TIME

    # _fea["SaccadeTimeShift"] = (np.argmax(_speeds) - (data.shape[0] // 2)) * EYE_SAMPLE_TIME

    # bias_res = compute_drop_point_bias_pred(
    #     eye_data=data,
    #     eye_index=data.index[np.argmax(_speeds)],
    #     ball_data=ball_data_df,
    #     ball_round=round_index
    # )
    # _fea["SaccadeDropBiasPred_Init"] = bias_res[0] * VR_SCALE
    # _fea["SaccadeDropBiasPred_Final"] = bias_res[1] * VR_SCALE

    # bias_res = compute_drop_point_bias_delay(
    #     eye_data=data,
    #     eye_index=data.index[np.argmax(_speeds)],
    #     ball_data=ball_data_df,
    #     ball_round=round_index
    # )
    # _fea["SaccadeDropBiasDelay_Init"] = bias_res[0] * VR_SCALE
    # _fea["SaccadeDropBiasDelay_Final"] = bias_res[1] * VR_SCALE
    return _fea

In [7]:
from utils import compute_dtw

def extract_trajectory_lite(round_index, data:pd.DataFrame, ball_data_df):
    _fea = {}
    
    _ball_df = ball_data_df[ball_data_df["round"]==round_index]
    _dtw = compute_dtw(
            line_1=data.loc[:, ["Screen.x", "Screen.y"]],
            line_2=_ball_df.loc[:, ["Ball.x", "Ball.y"]],
            # scale_to_percentage=True,
            # scale_metrics="start"
        )
    
    _fea["TrajectoryDTW"] = _dtw

    return _fea

## Main extrat features function

In [8]:
def extract_features_round(rounds:dict, data_df:pd.DataFrame, ball_data_df:pd.DataFrame):
    data_df["frame"] = data_df.index
    aligned_df = interplate_and_align(data_df, ball_data_df, EYE_SAMPLE_RATE, VIDEO_FPS, convert_dist=False)

    res = {}
    for round_index, round_indices in rounds.items():
        round_res = extract_saccade_features_lite(
            round_index,
            aligned_df.loc[round_indices, ["Screen.x", "Screen.y"]],
            aligned_df.loc[:, ["Ball.x", "Ball.y", "round"]]
            )
        
        traj_res = extract_trajectory_lite(
            round_index,
            aligned_df.loc[round_indices, ["Screen.x", "Screen.y"]],
            aligned_df.loc[:, ["Ball.x", "Ball.y", "round"]]
            )
        res[round_index] = {**round_res, **traj_res}
    return res

In [9]:
from utils import max_circle_radius

def threshold_find_match_round_dtw(eye_data:pd.DataFrame, ball_data_df:pd.DataFrame, order, scale_raw_data, mode="fast", dtw_th=1, dist_th=10):
    if mode=="fast":
        rounds, dtw_res = find_match_round_dtw_kmp(eye_data_df=eye_data, ball_data_df=ball_data_df, order=order, scale_raw_data=scale_raw_data)
    elif mode=="greedy":
        rounds, dtw_res = find_match_round_dtw(eye_data_df=eye_data, ball_data_df=ball_data_df, order=order, scale_raw_data=scale_raw_data)

    res = {}
    for _round, _round_index in rounds.items():
        if (max_circle_radius(eye_data.loc[_round_index, :]) >= dist_th) and (dtw_res[_round] <= dtw_th):
            res[_round] = _round_index

    return res

In [10]:
def extract_features(data, ball_data):
    data_df = pd.DataFrame(data).T
    data_df.ffill(inplace=True)
    data_df.bfill(inplace=True)

    ball_data_df = pd.DataFrame(ball_data)
    
    # match_rounds = label_round_hit(data_df.loc[:, ["Screen.x", "Screen.y"]], video_id)

    match_rounds = threshold_find_match_round_dtw(data_df.copy(), ball_data_df.copy(), order=0, scale_raw_data=True, mode="fast")
    # match_rounds = find_match_round_hit(data_df.loc[:, ["Screen.x", "Screen.y"]], video_id, time_range=7, dist=300)
    saccade_features = extract_features_round(match_rounds, data_df.copy(), ball_data_df.copy())
    
    return saccade_features

    # saccade_features = extract_saccade_features(match_rounds, data_df.loc[:, ["Screen.x", "Screen.y"]], video_id)
    # trajectory_features = extract_trajectory(data_df.loc[:, ["Screen.x", "Screen.y"]], video_id, scale_to_percentage=True)

    # return {**saccade_features, **trajectory_features}

# Extract Features of all Participants

In [11]:
all_people_fea = {}
video_list= []
for _person in all_data.keys():
    if not _person == "24071512_AD": continue
    _person_fea = {}

    for _video in all_data[_person].keys():
        # if not _video == "p7": continue
        if not _video in video_list: video_list.append(_video)
        
        fea_res = extract_features(data=all_data[_person][_video], ball_data=ball_data[_video.split("_")[0]])
        if fea_res:
            _person_fea[_video] = fea_res

    all_people_fea[_person] = _person_fea

In [12]:
def single_video_res(all_people_fea, video_id):
    res = {}
    for _p, person_fea in all_people_fea.items():
        try:
            for _r, _fea in person_fea[video_id].items():
                res[f"{_p}-{_r}"] = _fea
        except:
            continue
    return res


In [13]:
def single_person_res(all_people_fea, people_id):
    res = {}
    for _v, video_fea in all_people_fea[people_id].items():
        try:
            for _r, _fea in video_fea.items():
                res[f"{_v}-{_r}"] = _fea
        except:
            continue
    return res

In [14]:
person_fea = single_person_res(all_people_fea=all_people_fea, people_id="24071512_AD")
pd.DataFrame(person_fea).T.to_csv("24071512_AD_dtw_fast_rounds.csv")

In [15]:
video_id = video_list[0]
print(video_id)
video_res = single_video_res(all_people_fea=all_people_fea, video_id=video_id)
pd.DataFrame(video_res).T

w12


,SaccadeSpeed_Mean,SaccadeSpeed_Max,SaccadeSpeed_Min,SaccadeSpeed_Std,SaccadeAngel_Mean,SaccadeAngel_Max,SaccadeAngel_Min,SaccadeAngel_Std,SaccadeDelay,SaccadeDelayPercent,TrajectoryDTW
24071512_AD-1.0,171.719516,762.506229,0.0,270.898682,3.434390,15.250125,0.0,5.417974,1060.0,21.200000,442.116584
24071512_AD-2.0,507.940596,1523.821788,0.0,718.336480,10.158812,30.476436,0.0,14.366730,1460.0,28.627451,905.425931
24071512_AD-5.0,55.665336,444.931429,0.0,97.777255,1.113307,8.898629,0.0,1.955545,620.0,6.966292,268.773319
24071512_AD-9.0,196.943952,1070.342434,0.0,391.123187,3.938879,21.406849,0.0,7.822464,920.0,10.454545,643.512943
